In [ ]:
from hsd import HSD
from hsd_exodus import ExodusNetwork
from hsd_slayer import SlayerNetwork

In [ ]:
dataset = HSD(
    batch_size=128,
    encoding_dim=100,
    num_workers=4,
    download_dir="./data",
)
dataset.setup()
trainloader = dataset.train_dataloader()
testloader = dataset.val_dataloader()

In [ ]:
next(iter(trainloader))[0].shape

In [ ]:
from tqdm.auto import tqdm

def cycle_through_trainloader():
    for data, targets in tqdm(trainloader):
        data = data.cuda()
        targets = targets.cuda()

In [ ]:
cycle_through_trainloader()

In [ ]:
dict_args = dict(
    encoding_dim=100,
    n_hidden_layers=2,
    hidden_dim=128,
    tau_mem=100000.0,
    output_dim=20,
    spike_threshold=1.,
    learning_rate=1e-3,
    width_grad=1.,
    scale_grad=1.,
    decoding_func='max_over_time',
    optimizer="sgd",
)
slayer_model = SlayerNetwork(**dict_args, n_time_bins=250).cuda()
init_weights = slayer_model.state_dict()

exodus_model = ExodusNetwork(**dict_args, init_weights=init_weights).cuda()

sinabs_model = ExodusNetwork(**dict_args, init_weights=init_weights, backend='sinabs').cuda()


In [ ]:
from time import time
import numpy as np

algorithms = ["EXODUS", "SLAYER", "BPTT"]
times = {algo: {"forward": [], "backward": [], "reset": []} for algo in algorithms}

for algo, model in zip(["BPTT", "EXODUS"], [sinabs_model, exodus_model]):
    for i in tqdm(range(3)):
        times_epoch = {"forward": [], "backward": [], "reset": []}
        for data, target in tqdm(trainloader):        
            data = data.cuda()
            target = target.cuda()
            t0 = time()
            model.reset_states()
            t1 = time()
            y_hat = model(data)
            t2 = time()
            y_hat.sum().backward()
            t3 = time()
            times_epoch["reset"].append(t1-t0)
            times_epoch["forward"].append(t2-t1)
            times_epoch["backward"].append(t3-t2)
        for step, t in times_epoch.items():
            times[algo][step].append(np.mean(t))

In [ ]:
for i in tqdm(range(3)):
    t0 = time()
    times_epoch = {"forward": [], "backward": [], "reset": []}
    for data, target in tqdm(trainloader):
        data = data.cuda()
        target = target.cuda()
        t0 = time()
        y_hat = slayer_model(data)
        t1 = time()
        y_hat.sum().backward()
        t2 = time()
        times_epoch["forward"].append(t1-t0)
        times_epoch["backward"].append(t2-t1)
    for step, t in times_epoch.items():
        times["SLAYER"][step].append(np.mean(t))

In [ ]:
import numpy as np
for action in times["EXODUS"].keys():
    print(action)
    for model, ts in times.items():
        t = np.array(ts[action])
        print(f"\t{model}: ({np.mean(t):.2e} +- {np.std(t):.2e}) s")
        # np.save(f"timings_{model}.npy", t)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(times).to_csv("times_new.csv")

In [ ]:
# Convert previously saved csv to one line per measurement

times = pd.read_csv("times_new.csv", index_col=0)

table = [
    pd.DataFrame(
        {"algorithm": algo, "time": t, "step": step}
        for step in times.index
        for algo in times.loc[step].index
        for t in eval((times.loc[step].loc[algo]).replace("nan, ", "").replace("nan", ""))
    )
]
table = pd.concat(table, ignore_index=True)
table.to_csv("times_new_table.csv")